In [90]:
# imports
import pandas as pd
import numpy as np
import funciones as f
from geopy.geocoders import Nominatim

# rutas
homicidios_xlsx = 'data/homicidios.xlsx'
lesiones_xslx = 'data/lesiones.xlsx'

1. Lectura de datos

In [91]:
# Creamos un DataFrame para la hoja hechos del excel homicidios
df_homicidios_hechos = pd.read_excel(homicidios_xlsx, sheet_name='HECHOS')


In [92]:
df_homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [93]:
df_homicidios_hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


2. Descarte de columnas.
voy a ir descartando columnas en la medida en la que me asegure que la informacion que contengan no sea la faltante de alguna columna que si utilice.

Vemos si a la columna 'FECHA' le hace falta algo que podamos deducir de las 3 columnas contiguas.

In [94]:
# verificamos si la columna tiene algun valor nulo
if df_homicidios_hechos['FECHA'].isnull().any():
    print("La columna tiene al menos un valor nulo.")
else:
    print("La columna no tiene valores nulos.")

La columna no tiene valores nulos.


In [95]:
# verificamos que todos los datos son de tipo datetime
if f.columna_es_datetime(df_homicidios_hechos['FECHA']):
    print('Todos los datos de la columna FECHA son de tipo datetime')
else:
    print('Al menos un dato de la columna FECHA no es de tipo datetime')

Todos los datos de la columna FECHA son de tipo datetime


Como la columna fecha no tiene ningún valor faltante, ni nan, ni nat, ni null, y tampoco contiene datos que no sean especificamente datetime, podemos concluir que los datos son consistentes y puedo eliminar las columnas AAAA, MM, DD, debido a que me significan una redundancia en los datos.

In [96]:
# eliminamos las columnas AAAA, MM, DD
columnas_a_eliminar = ['AAAA', 'MM', 'DD']
df_homicidios_hechos = df_homicidios_hechos.drop(columnas_a_eliminar, axis=1)

vemos si a las columnas 'pos x' y 'pos y' les hace falta algo que podamos deducir de: 'LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Dirección Normalizada', 'XY (CABA)'

limpieza pos x

In [97]:
# al convertir una columna con datos de tipo latitud y longitud a float podremos ver que datos no satisfacen por lo menos ser numericos, estar separados por un punto, y que el primer caracter
# sea numerico o un simbolo negativo
try:
    df_homicidios_hechos['pos x'] = df_homicidios_hechos['pos x'].astype(float, errors='raise')
except ValueError as e:
    print(f"Error al convertir 'pos x' a tipo float: {e}")


Error al convertir 'pos x' a tipo float: could not convert string to float: '.'


Visualizamos datos en forma de '.' los cuales reemplazaremos con NaN

In [98]:
df_homicidios_hechos['pos x'] = df_homicidios_hechos['pos x'].replace('.', np.nan)

In [99]:
try:
    df_homicidios_hechos['pos x'] = df_homicidios_hechos['pos x'].astype(float, errors='raise')
    print('los datos han sido convertidos a tipo float correctamente')
except ValueError as e:
    print(f"Error al convertir 'pos x' a tipo float: {e}")

los datos han sido convertidos a tipo float correctamente


limpieza 'pos y'

In [100]:
try:
    df_homicidios_hechos['pos y'] = df_homicidios_hechos['pos y'].astype(float, errors='raise')
except ValueError as e:
    print(f"Error al convertir 'pos y' a tipo float: {e}")

Error al convertir 'pos y' a tipo float: could not convert string to float: '.'


visualizamos el mismo problema, repetimos los pasos

In [101]:
df_homicidios_hechos['pos y'] = df_homicidios_hechos['pos y'].replace('.', np.nan)

In [102]:
try:
    df_homicidios_hechos['pos y'] = df_homicidios_hechos['pos y'].astype(float, errors='raise')
except ValueError as e:
    print(f"Error al convertir 'pos y' a tipo float: {e}")

Una vez que los datos los tenemos, o en forma de float (y muy probablemente sean coordenadas validas) o en forma de NaN, procedemos a ver los registros que en 'pos x' y 'pox y' sean NaN, y si podemos rescatar esta info de las columnas que ya mencionamos anteriormente

In [103]:
# Filtrar el DataFrame para obtener las filas con NaN en ambas 'pos x' y 'pos y'
filas_con_nan = df_homicidios_hechos[df_homicidios_hechos['pos x'].isna() & df_homicidios_hechos['pos y'].isna()]

# Mostrar el resultado filtrando unicamente con los campos de los cuales podemos extraer info
filas_con_nan[['LUGAR_DEL_HECHO','TIPO_DE_CALLE','Calle','Altura','Dirección Normalizada','COMUNA','pos x','pos y']]

,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Dirección Normalizada,COMUNA,pos x,pos y
38,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,13,NaN,NaN
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,4,NaN,NaN
119,SD,CALLE,NaN,NaN,NaN,0,NaN,NaN
139,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,NaN,AUTOPISTA 25 DE MAYO,0,NaN,NaN
176,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",NaN,"LUGONES, LEOPOLDO AV.",14,NaN,NaN
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN,NaN,9,NaN,NaN
181,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN,NaN,7,NaN,NaN
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,NaN,NaN
313,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,14,NaN,NaN
546,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,14,NaN,NaN


In [104]:
geolocalizador = Nominatim(user_agent="mi_aplicacion")
direccion = "MURGUIONDO 2700"

ubicacion = geolocalizador.geocode(direccion)

if ubicacion:
    print(f"Latitud: {ubicacion.latitude}, Longitud: {ubicacion.longitude}")
else:
    print("No se pudo obtener la latitud y longitud.")

Latitud: -34.66337, Longitud: -58.4932488
